In [2]:
#Getting the data from the local file
import numpy as np
import pandas as pd
from datetime import date
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from scipy import stats

df=pd.read_csv('D:\CHARIT UMESH MANI\employees.csv')
df

,Agency,Pay Class Category,Pay Scale Type,Position Number,Position Title,Extra Help Flag,Class Code,Employee Name,Career Service Date,Pay Grade,Gender,Race,Percent of Time,Annual Salary
0,AR DEPT OF VETERAN AFFAIRS,MEDICAL PROFESSIONAL,CLASSIFIED,22164498,REGISTERED NURSE COORDINATOR,NaN,L019C,ARNETTA L BRANDON,08-06-2019,MP02,FEMALE,BLACK/NOT HISPANIC ORIGIN,100,81096.70
1,LEGISLATIVE AUDIT,SENIOR EXECUTIVE,CLASSIFIED,22097040,LA AUDIT MANAGER,NaN,Q006N,A S ADAMS CPA,06-01-1992,SE02,MALE,WHITE/NOT HISPANIC ORIGIN,100,129499.97
2,DAH - CENTRAL ADMINISTRATION,GENERAL SERVICES,CLASSIFIED,22092728,DAH PROGRAM COORDINATOR,NaN,G186C,A. ABIGAIL HOWE,22-07-1979,GS06,FEMALE,WHITE/NOT HISPANIC ORIGIN,100,56051.01
3,DFA - REVENUE SERVICES DIV,GENERAL SERVICES,CLASSIFIED,22133966,TAX INVESTIGATOR,NaN,X172C,A. CAROL DODSON,16-03-2018,GS03,FEMALE,WHITE/NOT HISPANIC ORIGIN,100,26033.90
4,DFA - OFC OF CHILD SUPPORT ENF,GENERAL SERVICES,CLASSIFIED,22111298,DFA SERVICE REPRESENTATIVE,NaN,C059C,AALIYAH C. MCSHANE,16-06-2019,GS03,FEMALE,BLACK/NOT HISPANIC ORIGIN,100,26033.90
5,AR DEPT OF HUMAN SERVICES,GENERAL SERVICES,CLASSIFIED,22151035,RESIDENTIAL CARE TECHNICIAN,NaN,M083C,AALIYAH K. SIMMONS,22-05-2017,GS02,FEMALE,BLACK/NOT HISPANIC ORIGIN,100,25028.22
6,AR DEPT OF HUMAN SERVICES,GENERAL SERVICES,CLASSIFIED,22101529,RESIDENTIAL CARE TECHNICIAN,NaN,M083C,AALIYAH O. JOHNSON,06-05-2019,GS02,FEMALE,BLACK/NOT HISPANIC ORIGIN,100,24199.76
7,AR DEPT OF HUMAN SERVICES,GENERAL SERVICES,CLASSIFIED,22100547,RESIDENTIAL CARE ASSISTANT,NaN,M089C,AALIYAH P. BECK,23-11-2019,GS01,FEMALE,WHITE/NOT HISPANIC ORIGIN,100,21999.95
8,DEPARTMENT OF CORRECTION,GENERAL SERVICES,CLASSIFIED,22084849,ADC/DCC CORPORAL,NaN,T075C,AALIYAH S HARDEN,09-03-2020,GS05,FEMALE,BLACK/NOT HISPANIC ORIGIN,100,32404.94
9,DEPARTMENT OF CORRECTION,GENERAL SERVICES,CLASSIFIED,22084987,ADC/DCC CORPORAL,NaN,T075C,AALIYAH T FRANKLIN,09-03-2020,GS05,FEMALE,BLACK/NOT HISPANIC ORIGIN,100,32404.94


In [3]:
#Changing string data into numarical data
df1=df[['Agency', 'Pay Class Category', 'Pay Scale Type','Position Title','Class Code',
        'Career Service Date', 'Pay Grade', 'Gender','Race','Extra Help Flag']].copy()
df2=df1.copy()
for i in df2.columns:
    p=set(df2[i])
    p=list(p)
    for j,k in enumerate(p):
        df1[i]=df1[i].replace({k:(j+1)*10})
df1              
   
        




,Agency,Pay Class Category,Pay Scale Type,Position Title,Class Code,Career Service Date,Pay Grade,Gender,Race,Extra Help Flag
0,390,50,30,21190,5250,71130,1760,30,40,10
1,60,30,30,1890,9060,47330,1380,10,60,10
2,220,40,30,9730,17910,28820,510,30,60,10
3,1000,40,30,9840,14620,50230,1110,30,60,10
4,1050,40,30,13870,5210,60900,1110,30,40,10
5,380,40,30,22120,4520,54160,2370,30,40,10
6,380,40,30,22120,4520,37500,2370,30,40,10
7,380,40,30,10330,8180,34700,590,30,60,10
8,1390,40,30,22150,14910,47040,1500,30,40,10
9,1390,40,30,22150,14910,47040,1500,30,40,10


In [4]:
# getting dates into no. of days(in career service date) and check for null values  
d=df2.columns
df2[d[5]]=pd.to_datetime(df2[d[5]],errors="coerce",dayfirst=True)
Days=[]
for i in range(len(df2[d[5]])):
    k=df2[d[5]][i]
    delta=date.today()-k.date()
    k=delta.days
    Days.append(k)
Days=pd.Series(Days)
Days=Days.fillna(value=0)
df1[d[5]]=Days
print(df1.head())
#checking null values
for q in range(len(d)):
    k1=df1[d[q]]
    nck=pd.isna(k1)
    k=set(nck)
    print(q,k,"         ",d[q])


   Agency  Pay Class Category  Pay Scale Type  Position Title  Class Code  \
0     390                  50              30           21190        5250   
1      60                  30              30            1890        9060   
2     220                  40              30            9730       17910   
3    1000                  40              30            9840       14620   
4    1050                  40              30           13870        5210   

   Career Service Date  Pay Grade  Gender  Race  Extra Help Flag  
0                352.0       1760      30    40               10  
1              10367.0       1380      10    60               10  
2              14918.0        510      30    60               10  
3                801.0       1110      30    60               10  
4                344.0       1110      30    40               10  
0 {False}           Agency
1 {False}           Pay Class Category
2 {False}           Pay Scale Type
3 {False}           Position Title

In [5]:
#Shuffled and splited the data into training and test data , and changing them into Tensors
df1['Annual Salary']=((df['Annual Salary'])/1000).copy()

# Defined a size for  train set 
train_size = int(0.7 * len(df1))

# Splitting  dataset 
train_set = df1[:train_size]
test_set = df1[train_size:]

#saperated the input values and target values

X1=train_set[['Agency', 'Pay Class Category', 'Pay Scale Type', 'Position Title',
       'Class Code', 'Career Service Date', 'Pay Grade', 'Gender', 'Race',
       'Extra Help Flag']]
print(X1.size)
y1=train_set['Annual Salary']
print(y1.size)
X1=torch.Tensor(X1.values)
y1=torch.Tensor(y1.values)
X2=test_set[['Agency', 'Pay Class Category', 'Pay Scale Type', 'Position Title',
       'Class Code', 'Career Service Date', 'Pay Grade', 'Gender', 'Race',
       'Extra Help Flag']]
y2=test_set['Annual Salary']
X2=torch.Tensor(X2.values)
y2=torch.Tensor(y2.values)

227750
22775


In [ ]:
# Deeplearning Code and Passing through code
#code
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 12)
        self.fc2 = nn.Linear(12, 18)
        self.fc3 = nn.Linear(18, 12)
        self.fc4 = nn.Linear(12, 10)
        self.fc5 = nn.Linear(10, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.fc5(x)
        return x 

net = Net()
loss_function = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001)

for epoch in range(100):
    for data in zip(X1,y1):
        X, y =data[0],data[1]
        #print(X,y)
        net.zero_grad()
        output = net(X)
        #print(output,y)# to check the size(not requried)
        output.size=y.size
        #print(y-output)      
        loss = F.mse_loss(output, y)
        #print(loss)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(net.parameters(),0.1)
        optimizer.step()
    print(loss)
    if loss<=0.03:
        with torch.no_grad():
            for data in zip(X2,y2):
                X, y = data[0],data[1]
                output = net(X)
                print(output)
                print(output-y)
        break
        

tensor(200.0908, grad_fn=<MseLossBackward>)
tensor(199.1732, grad_fn=<MseLossBackward>)
tensor(175.0893, grad_fn=<MseLossBackward>)
tensor(177.1032, grad_fn=<MseLossBackward>)
tensor(168.4399, grad_fn=<MseLossBackward>)
tensor(136.8312, grad_fn=<MseLossBackward>)
tensor(128.2905, grad_fn=<MseLossBackward>)
tensor(130.7590, grad_fn=<MseLossBackward>)
tensor(127.9432, grad_fn=<MseLossBackward>)
tensor(133.8162, grad_fn=<MseLossBackward>)
tensor(138.3843, grad_fn=<MseLossBackward>)
tensor(139.0696, grad_fn=<MseLossBackward>)
tensor(129.3958, grad_fn=<MseLossBackward>)
tensor(129.7291, grad_fn=<MseLossBackward>)
tensor(131.8016, grad_fn=<MseLossBackward>)
tensor(132.1246, grad_fn=<MseLossBackward>)
tensor(123.3701, grad_fn=<MseLossBackward>)
tensor(123.1677, grad_fn=<MseLossBackward>)
tensor(121.6517, grad_fn=<MseLossBackward>)
tensor(118.7266, grad_fn=<MseLossBackward>)
tensor(115.5238, grad_fn=<MseLossBackward>)
tensor(113.0107, grad_fn=<MseLossBackward>)
tensor(115.1906, grad_fn=<MseLos

In [ ]:
E=[]
AV=[]
PV=[]
DFF=[]
with torch.no_grad():
            for data in zip(X2,y2):
                X, y = data[0],data[1]
                output = net(X)
                err = output-y
                err = round(float(err),4)
                diffr = err/y
                AV.append(float(y))
                PV.append(float(output))
                E.append(float(err))
                DFF.append(float(diffr))
AV=pd.Series(AV)
PV=pd.Series(PV)
E=pd.Series(E)
DFF=pd.Series(DFF)
df3=pd.DataFrame(columns=["actual value","predication value","error","difference ratio"])
df3["actual value"]=AV
df3["predication value"]=PV
df3["error"]=E
df3["difference ratio"]=DFF
df3.to_excel('employeesdata2.xlsx', sheet_name='Sheet2')